In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 1.6 MB/s 


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
import numpy as np
import os
import numpy as np
import skimage
#import skimage.io as io
import skimage.transform as transform
import torch

import numpy as np
import pandas as pd
import torch
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import cv2
import matplotlib.pyplot as plt
import skimage 

#import logging
#import traceback
from typing import Optional, Tuple


def kits_normalization(input_image: np.ndarray):
    # first, clip to [-62, 310] (corresponds to 0.5 and 99.5 percentile in the foreground regions)
    # then, subtract 104.9 and divide by 75.3 (corresponds to mean and std in the foreground regions, respectively)
    clip_min = -62
    clip_max = 301
    mean_val = 104.0
    std_val = 75.3
    input_image = np.minimum(np.maximum(input_image, clip_min), clip_max)
    input_image -= mean_val
    input_image /= std_val
    return input_image

def normalize_to_range(input_image: np.ndarray, range: Tuple = (0.0, 1.0)):
    """
    Scales tensor to range
    @param input_image: image of shape (H x W x C)
    @param range:       bounds for normalization
    @return:            normalized image
    """
    max_val = input_image.max()
    min_val = input_image.min()
    if min_val == max_val == 0:
        return input_image
    input_image = input_image - min_val
    input_image = input_image / (max_val - min_val)
    input_image = input_image * (range[1] - range[0])
    input_image = input_image + range[0]
    return input_image

from typing import Iterable

def center_crop(np_image: np.ndarray,
                new_shape: Iterable[int],
                outside_val: float = 0
                ) -> np.ndarray:
    output_image = np.full(new_shape, outside_val, np_image.dtype)

    slices = tuple()
    offsets = tuple()
    for it, sh in enumerate(new_shape):
        size = sh // 2
        if it == 0:
            center = np_image.shape[it] - size
        else:
            center = (np_image.shape[it] // 2)
        start = center - size
        stop = center + size + (sh % 2)

        # computing what area of the original image will be in the cropped output
        slce = slice(max(0, start), min(np_image.shape[it], stop))
        slices += (slce,)

        # computing offset to pad if the crop is partly outside of the scan
        offset = slice(-min(0, start), 2 * size - max(0, (start + 2 * size) - np_image.shape[it]))
        offsets += (offset,)

    output_image[offsets] = np_image[slices]

    return output_image


def pad_image(image: np.ndarray, outer_height: int, outer_width: int, pad_value: Tuple):
    """
    Pastes input image in the middle of a larger one
    @param image:        image of shape (H x W x C)
    @param outer_height: final outer height
    @param outer_width:  final outer width
    @param pad_value:    value for padding around inner image
    @return:             padded image
    """
    inner_height, inner_width = image.shape[0], image.shape[1]
    h_offset = int((outer_height - inner_height) / 2.0)
    w_offset = int((outer_width - inner_width) / 2.0)
    outer_image = np.ones((outer_height, outer_width, 3), dtype=image.dtype) * pad_value
    outer_image[h_offset:h_offset + inner_height, w_offset:w_offset + inner_width, :] = image

    return outer_image


class deeeeset(Dataset):
    def __init__(self,in_out):
        self.in_out=in_out
        
        #self.new_shape=new_shape
        #self.input_data = input_data
        self.ids=self.in_out['SubjectId']
        self.label=self.in_out['task_2_label']
        
        self.feature=self.in_out.drop(['SubjectId','gender','aua_risk_group','task_1_label','task_2_label'],axis=1, inplace=True)

    
            
        
    def __getitem__(self,idx):
        
        feat=self.in_out.iloc[idx]
        x_feature=pd.DataFrame(feat).T
        
        # norm_img=get_array_img
        
        y_output=self.label[idx]
        x_f_array=np.array(x_feature)
        x_f_array_t=torch.from_numpy(x_f_array).float()
        x_f_array_t=torch.squeeze(x_f_array_t,axis=0)
        return (x_f_array_t,y_output)
    
    def __len__(self):
        return(len(self.label))
    


#input_image = center_crop(input_image, new_shape=new_shape) 
    
    
filet=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/trainf4new.csv')
#data='/home/imranr/knights/data'

filev=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/validf4new.csv')




#len(filev['task_1_label']==0)
#len(filev['task_1_label']==1)
#len(filev['task_1_label']==2)
#len(filev['task_1_label']==3)
#len(filev['task_1_label']==4)

#classes=['NoAT','CanAT']
#c1=filet['task_1_label'].value_counts()[0]
#175
c1=filet['task_2_label'].value_counts()[0]
c2=filet['task_2_label'].value_counts()[1]
c3=filet['task_2_label'].value_counts()[2]
c4=filet['task_2_label'].value_counts()[3]
c5=filet['task_2_label'].value_counts()[4]
print(c1)
print(c2)
print(c3)
print(c4)
print(c5)
#c2=filet['task_1_label'].value_counts()[1]
#65

train_dataset=deeeeset(filet)
valid_dataset=deeeeset(filev)

#print(len(train_dataset))
#print(len(valid_dataset))

f,o=valid_dataset[2]

#print(i.shape)
#print(i.min())
#print(i.max())
#print(o)
#print(f)

# i,f, o=train_dataset[2]

# print(i.shape)
# print(i.min())
# print(i.max())
# print(o)
# print(f)

#%
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(ob_d, batch_size=4, shuffle=True)
# #test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
# for i, data in enumerate(train_dataloader):
#     img,f,l=data
#     print(img.shape)
#     print(f.shape)
#     print(l)
#     break
    

batch=4

train_dataloader = DataLoader(train_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=True)

valid_dataloader = DataLoader(valid_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=False)


20
112
37
34
37


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
cd /content/drive/MyDrive/knight_challenegs2022/models

/content/drive/MyDrive/knight_challenegs2022/models


In [ ]:
!mkdir fold0modelt2

In [ ]:
!mkdir fold1modelt2

In [ ]:
!mkdir fold2modelt2

In [ ]:
!mkdir fold3modelt2

In [ ]:
!mkdir fold4modelt2

In [ ]:
cd /content/drive/MyDrive/knight_challenegs2022

/content/drive/MyDrive/knight_challenegs2022


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
import numpy as np
import os
import numpy as np
import skimage
#import skimage.io as io
import skimage.transform as transform
import torch

import numpy as np
import pandas as pd
import torch
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import cv2
import matplotlib.pyplot as plt
import skimage 

#import logging
#import traceback
from typing import Optional, Tuple


def kits_normalization(input_image: np.ndarray):
    # first, clip to [-62, 310] (corresponds to 0.5 and 99.5 percentile in the foreground regions)
    # then, subtract 104.9 and divide by 75.3 (corresponds to mean and std in the foreground regions, respectively)
    clip_min = -62
    clip_max = 301
    mean_val = 104.0
    std_val = 75.3
    input_image = np.minimum(np.maximum(input_image, clip_min), clip_max)
    input_image -= mean_val
    input_image /= std_val
    return input_image

def normalize_to_range(input_image: np.ndarray, range: Tuple = (0.0, 1.0)):
    """
    Scales tensor to range
    @param input_image: image of shape (H x W x C)
    @param range:       bounds for normalization
    @return:            normalized image
    """
    max_val = input_image.max()
    min_val = input_image.min()
    if min_val == max_val == 0:
        return input_image
    input_image = input_image - min_val
    input_image = input_image / (max_val - min_val)
    input_image = input_image * (range[1] - range[0])
    input_image = input_image + range[0]
    return input_image

from typing import Iterable

def center_crop(np_image: np.ndarray,
                new_shape: Iterable[int],
                outside_val: float = 0
                ) -> np.ndarray:
    output_image = np.full(new_shape, outside_val, np_image.dtype)

    slices = tuple()
    offsets = tuple()
    for it, sh in enumerate(new_shape):
        size = sh // 2
        if it == 0:
            center = np_image.shape[it] - size
        else:
            center = (np_image.shape[it] // 2)
        start = center - size
        stop = center + size + (sh % 2)

        # computing what area of the original image will be in the cropped output
        slce = slice(max(0, start), min(np_image.shape[it], stop))
        slices += (slce,)

        # computing offset to pad if the crop is partly outside of the scan
        offset = slice(-min(0, start), 2 * size - max(0, (start + 2 * size) - np_image.shape[it]))
        offsets += (offset,)

    output_image[offsets] = np_image[slices]

    return output_image


def pad_image(image: np.ndarray, outer_height: int, outer_width: int, pad_value: Tuple):
    """
    Pastes input image in the middle of a larger one
    @param image:        image of shape (H x W x C)
    @param outer_height: final outer height
    @param outer_width:  final outer width
    @param pad_value:    value for padding around inner image
    @return:             padded image
    """
    inner_height, inner_width = image.shape[0], image.shape[1]
    h_offset = int((outer_height - inner_height) / 2.0)
    w_offset = int((outer_width - inner_width) / 2.0)
    outer_image = np.ones((outer_height, outer_width, 3), dtype=image.dtype) * pad_value
    outer_image[h_offset:h_offset + inner_height, w_offset:w_offset + inner_width, :] = image

    return outer_image


class deeeeset(Dataset):
    def __init__(self,in_out):
        self.in_out=in_out
        
        #self.new_shape=new_shape
        #self.input_data = input_data
        self.ids=self.in_out['SubjectId']
        self.label=self.in_out['task_2_label']
        
        self.feature=self.in_out.drop(['SubjectId','gender','aua_risk_group','task_1_label','task_2_label'],axis=1, inplace=True)

    
            
        
    def __getitem__(self,idx):
        
        feat=self.in_out.iloc[idx]
        x_feature=pd.DataFrame(feat).T
        
        # norm_img=get_array_img
        
        y_output=self.label[idx]
        x_f_array=np.array(x_feature)
        x_f_array_t=torch.from_numpy(x_f_array).float()
        x_f_array_t=torch.squeeze(x_f_array_t,axis=0)
        return (x_f_array_t,y_output)
    
    def __len__(self):
        return(len(self.label))
    


#input_image = center_crop(input_image, new_shape=new_shape) 
    
    
filet=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/trainf4new.csv')
#data='/home/imranr/knights/data'

filev=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/validf4new.csv')




#len(filev['task_1_label']==0)
#len(filev['task_1_label']==1)

#classes=['NoAT','CanAT']
#c1=filet['task_1_label'].value_counts()[0]
#175

#c2=filet['task_1_label'].value_counts()[1]
#65

train_dataset=deeeeset(filet)
valid_dataset=deeeeset(filev)

print(len(train_dataset))
print(len(valid_dataset))

f,o=valid_dataset[2]

#print(i.shape)
#print(i.min())
#print(i.max())
print(o)
print(f)

# i,f, o=train_dataset[2]

# print(i.shape)
# print(i.min())
# print(i.max())
# print(o)
# print(f)

#%
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(ob_d, batch_size=4, shuffle=True)
# #test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
# for i, data in enumerate(train_dataloader):
#     img,f,l=data
#     print(img.shape)
#     print(f.shape)
#     print(l)
#     break
    

batch=4

train_dataloader = DataLoader(train_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=True)

valid_dataloader = DataLoader(valid_dataset, batch_size= batch,pin_memory=True,num_workers=6, shuffle=False)


import sys 
import os
import glob
import time
import random
import os
import glob
import time
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Copyright (c) MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
(C) Copyright 2021 IBM Corp.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
Created on June 30, 2021
"""

from typing import Tuple, Any

import torch.nn as nn
from torch import Tensor
from torch.hub import load_state_dict_from_url
from torchvision.models.video.resnet import VideoResNet, BasicBlock, Conv3DSimple, BasicStem, model_urls


class FuseBackboneResnet3D(VideoResNet):
    """
    3D model classifier (ResNet architecture"
    """

    def __init__(self, pretrained: bool = False, in_channels: int = 1, name: str = "r3d_18") -> None:
        """
        Create 3D ResNet model
        :param pretrained: Use pretrained weights
        :param in_channels: number of input channels
        :param name: model name. currently only 'r3d_18' is supported
        """
        # init parameters per required backbone
        init_parameters = {
            'r3d_18': {'block': BasicBlock,
                       'conv_makers': [Conv3DSimple] * 4,
                       'layers': [2, 2, 2, 2],
                       'stem': BasicStem},
        }[name]
        # init original model
        super().__init__(**init_parameters)

        # load pretrained parameters if required
        if pretrained:
            state_dict = load_state_dict_from_url(model_urls[name])
            self.load_state_dict(state_dict)

        # save input parameters
        self.pretrained = pretrained
        self.in_channels = in_channels
        # override the first convolution layer to support any number of input channels
        self.stem = nn.Sequential(
            nn.Conv3d(self.in_channels, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2),
                      padding=(1, 3, 3), bias=False),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )

    def features(self, x: Tensor) -> Any:
        """
        Extract spatial features - given a 3D tensor
        :param x: Input tensor - shape: [batch_size, channels, z, y, x]
        :return: spatial features - shape [batch_size, n_features, z', y', x']
        """
        x = self.stem(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, x: Tensor) -> Tuple[Tensor, None, None, None]:  # type: ignore
        """
        Forward pass. 3D global classification given a volume
        :param x: Input volume. shape: [batch_size, channels, z, y, x]
        :return: logits for global classification. shape: [batch_size, n_classes].
        """
        x = self.features(x)
        return x
    
backbone=FuseBackboneResnet3D(pretrained=True)   
from typing import Optional, Sequence
import torch.nn as nn

class ClassifierMLP(nn.Module):
    def __init__(self, in_ch: int, num_classes: Optional[int], layers_description: Sequence[int]=(256,128), dropout_rate: float = 0.1):
        super().__init__()
        layer_list = []
        layer_list.append(nn.Linear(in_ch, layers_description[0]))
        layer_list.append(nn.ReLU())
        if dropout_rate is not None and dropout_rate > 0:
            layer_list.append(nn.Dropout(p=dropout_rate))
        last_layer_size = layers_description[0]
        for curr_layer_size in layers_description[1:]:
            layer_list.append(nn.Linear(last_layer_size, curr_layer_size))
            layer_list.append(nn.ReLU())
            if dropout_rate is not None and dropout_rate > 0:
                layer_list.append(nn.Dropout(p=dropout_rate))
            last_layer_size = curr_layer_size
        
        if num_classes is not None:
            layer_list.append(nn.Linear(last_layer_size, num_classes))
        
        self.classifier = nn.Sequential(*layer_list)

    def forward(self, x):
        x = self.classifier(x)
        return x
  
model=ClassifierMLP(12,5)  
import torch    

#inp=torch.rand(1,10)
#out=model(inp)
#print(out.shape)


#model=nn.DataParallel(model)
model=model.to(device)
#loss_func=nn.BCEWithLogitsLoss()
from tqdm import tqdm
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.00001)

#classes=['NRG','RG']
#classes=['NoAT','CanAT']
classes=['B','LR','IR','HR','VHR']
#classes=[B-score	LR-score	IR-score	HR-score	VHR-score]
#c1=data['label'].value_counts()[0]
#c2=data['label'].value_counts()[1]
my_distribution=np.array([20,112,37,34,37])
class_weights = torch.from_numpy(np.divide(1, my_distribution)).float().to(device)
class_weights = class_weights / class_weights.sum()
for i, c in enumerate(classes):
  print('Weight for class %s: %f' % (c, class_weights.cpu().numpy()[i]))
loss_func = nn.CrossEntropyLoss(weight=class_weights)
#label=label.to(torch.int64)
################################ training functions ###################
def train_fn(model,train_loader):
    model.train()
    counter=0
    training_run_loss=0.0
    train_running_correct=0.0
    for i, data in tqdm(enumerate(train_loader),total=int(len(train_dataset)/train_loader.batch_size)):
        counter+=1
        # extract dataset
        feature,label=data
        #imge=imge.float()
        #label=label.float()
        label.to(torch.int64)
        feature=feature.float()
        feature=feature.to(device)
        #imge=imge.to(device)
        label=label.to(device)
        #imge=imge.cuda()
        #label=label.cuda()
        # zero_out the gradient
        optimizer.zero_grad()
        output=model(feature)
        loss=loss_func(output,label)
        training_run_loss+=loss.item()
        _,preds=torch.max(output.data,1)
        train_running_correct+=(preds==label).sum().item()
        loss.backward()
        optimizer.step()
    ###################### state computation ###################
    train_loss=training_run_loss/len(train_loader.dataset)
    train_loss_ep.append(train_loss)
    train_accuracy=100.* train_running_correct/len(train_loader.dataset)
    train_accuracy_ep.append(train_accuracy)
    print(f"Train Loss:{train_loss:.4f}, Train Acc:{train_accuracy:0.2f}")
    return train_loss_ep,train_accuracy_ep

########################## validation function ##################
def validation_fn(model,valid_loader):
  # evluation start
    print("validation start")
    
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    with torch.no_grad():
        for i,data in tqdm(enumerate(valid_loader),total=int(len(valid_loader)/valid_loader.batch_size)):
            feature,label=data
            #imge=imge.float()
            #label=label.float()
            label.to(torch.int64)
            feature=feature.float()
            feature=feature.to(device)
            #imge=imge.to(device)
            label=label.to(device)
            #imge=imge.cuda()
            #label=label.cuda()
            output=model(feature)
            loss=loss_func(output,label)
            val_running_loss+=loss.item()
            _,pred=torch.max(output.data,1)
            val_running_correct+=(pred==label).sum().item()
        val_loss=val_running_loss/len(valid_loader.dataset)
        val_loss_ep.append(val_loss)
        val_accuracy=100.* val_running_correct/(len(valid_loader.dataset))
        val_accuracy_ep.append(val_accuracy)
        print(f"Val Loss:{val_loss:0.4f}, Val_Acc:{val_accuracy:0.2f}")
        return val_loss_ep,val_accuracy_ep

import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=0.0001)
train_loss_ep=[]
train_accuracy_ep=[]
val_loss_ep=[]
val_accuracy_ep=[]
lr = 3e-4
log = pd.DataFrame(index=[], columns=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])
early_stop=20
epochs=500
best_acc = 0
name='fold4modelt2'
trigger = 0
for epoch in range(epochs):
    print('Epoch [%d/%d]' %(epoch, epochs))
    # train for one epoch
    train_loss_ep,train_accuracy_ep=train_fn(model,train_dataloader)
    train_loss_ep1=np.mean(train_loss_ep)
    train_accuracy_ep1=np.mean(train_accuracy_ep)
    #y_pred,labels=Prediciton_fn(model,valid_loader)

    val_loss_ep,val_accuracy_ep=validation_fn(model,valid_dataloader)
    val_loss_ep1=np.mean(val_loss_ep)
    val_accuracy_ep1=np.mean(val_accuracy_ep)
    
    print('loss %.4f - accu %.4f - val_loss %.4f - val_accu %.4f'%(train_loss_ep1, train_accuracy_ep1, val_loss_ep1, val_accuracy_ep1))

    tmp = pd.Series([epoch,lr,train_loss_ep1,train_accuracy_ep1,val_loss_ep1,val_accuracy_ep1], index=['epoch', 'lr', 'loss', 'accu', 'val_loss', 'val_accu'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv('models/%s/log.csv' %name, index=False)

    trigger += 1

    if val_accuracy_ep1 > best_acc:
        torch.save(model.state_dict(), 'models/%s/3dmodeltask2f3.pth' %name)
        best_acc = val_accuracy_ep1
        print("=> saved best model")
        trigger = 0

    # early stopping
    if not early_stop is None:
        if trigger >= early_stop:
            print("=> early stopping")
            break

    torch.cuda.empty_cache() 

240
60
1
tensor([66.0000, 30.6900,  1.0000,  0.0000,  0.0000,  2.3000, 80.0000,  3.0000,
         1.0000,  0.8340,  0.8340,  0.5000])


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Weight for class B: 0.351137
Weight for class LR: 0.062703
Weight for class IR: 0.189804
Weight for class HR: 0.206551
Weight for class VHR: 0.189804
Epoch [0/500]


100%|██████████| 60/60 [00:00<00:00, 113.20it/s]

Train Loss:0.6117, Train Acc:27.50
validation start


...

Val Loss:0.3499, Val_Acc:46.67
loss 0.3460 - accu 48.3121 - val_loss 0.3549 - val_accu 49.7279
Epoch [196/500]


100%|██████████| 60/60 [00:00<00:00, 104.07it/s]

Train Loss:0.3084, Train Acc:53.75
validation start



15it [00:00, 49.04it/s]


Val Loss:0.3526, Val_Acc:45.00
loss 0.3458 - accu 48.3397 - val_loss 0.3549 - val_accu 49.7039
=> early stopping


In [ ]:
#case_id	
##B-score	
#LR-score	
#IR-score	
#HR-score	
#VHR-score

In [ ]:
import pandas as pd

pathtest=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/test_featurelatest.csv')
#pathtest.

In [ ]:
import torch
pathmodel='/content/drive/MyDrive/knight_challenegs2022/models/fold4modelt2/3dmodeltask2f3.pth'
trainedmodel=torch.load(pathmodel)
model.load_state_dict(trainedmodel)
model.eval()
model.to(device)

ClassifierMLP(
  (classifier): Sequential(
    (0): Linear(in_features=12, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=128, out_features=5, bias=True)
  )
)

In [ ]:
#NoAT-score
#CanAT-score
#case_id
import torch
#pathmodel='/content/drive/MyDrive/knight_challenegs2022/models/fold0model/3dmodeltask1.pth'
#trainedmodel=torch.load(pathmodel)
#model.load_state_dict(trainedmodel)
model.eval()
model.to(device)
dataframe={'case_id':[],
           'NoAT-score':[],
           'CanAT-score':[]}
CLINICAL_NAMES=['case_id','B-score','LR-score',
                'IR-score','HR-score',
                'VHR-score']

df = pd.DataFrame(columns=CLINICAL_NAMES)
for i,patinet in enumerate(pathtest['SubjectId']):
  #pat=patinet.split('_')[-1]
  #pat=str(0000)+i
  number_str = str(i)
  pat = number_str.zfill(5)
  #print(i)
  #print(patinet)
  features=pathtest.iloc[i]
  feature=features.drop(['SubjectId'])
  feature1=feature.drop(['gender'])
  #print(feature1.shape)
  x_f_array=np.array(feature1).astype(float)
  #print(x_f_array)
  #.astype(int)
  x_f_array_t=torch.from_numpy(x_f_array).float()
  x_f_array_t=torch.unsqueeze(x_f_array_t,axis=0).to(device)
  prediction=model(x_f_array_t)
  output=torch.softmax(prediction, dim=1)
  output=torch.squeeze(output,axis=0)
  pred=output.detach().cpu().numpy()
  #print(output.detach().cpu().numpy())
  df.loc[i,'case_id']=pat
  df.loc[i,'B-score']='%.1f' % pred[0]
  df.loc[i,'LR-score']='%.1f' % pred[1]
  df.loc[i,'IR-score']='%.1f' % pred[2]
  df.loc[i,'HR-score']='%.1f' % pred[3]
  df.loc[i,'VHR-score']='%.1f' % pred[4]
  #case_id	
  ##B-score	
  #LR-score	
  #IR-score	
  #HR-score	
  #VHR-score
  #dataframe['case_id'].append(str(pat))
  #dataframe['NoAT-score'].append(pred[0])
  #dataframe['CanAT-score'].append(pred[1])
  #features.iloc[i]
  #print(x_f_array_t.shape)
  print(pat)

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042
00043
00044
00045
00046
00047
00048
00049
00050
00051
00052
00053
00054
00055
00056
00057
00058
00059
00060
00061
00062
00063
00064
00065
00066
00067
00068
00069
00070
00071
00072
00073
00074
00075
00076
00077
00078
00079
00080
00081
00082
00083
00084
00085
00086
00087
00088
00089
00090
00091
00092
00093
00094
00095
00096
00097
00098
00099
00100
00101
00102


In [ ]:
df['case_id']=df['case_id'].str.rjust(2, "0")

In [ ]:
df

,case_id,B-score,LR-score,IR-score,HR-score,VHR-score
0,00000,0.0,0.1,0.2,0.3,0.4
1,00001,0.4,0.4,0.1,0.1,0.0
2,00002,0.4,0.4,0.1,0.1,0.0
3,00003,0.0,1.0,0.0,0.0,0.0
4,00004,0.1,0.1,0.3,0.3,0.2
...,...,...,...,...,...,...
98,00098,0.1,0.1,0.2,0.4,0.2
99,00099,0.1,0.4,0.3,0.1,0.1
100,00100,0.0,1.0,0.0,0.0,0.0
101,00101,0.1,0.7,0.1,0.0,0.0


In [ ]:
df['case_id'] = df['case_id'].apply('="{}"'.format)

In [ ]:
df

,case_id,B-score,LR-score,IR-score,HR-score,VHR-score
0,"=""00000""",0.0,0.1,0.2,0.3,0.4
1,"=""00001""",0.4,0.4,0.1,0.1,0.0
2,"=""00002""",0.4,0.4,0.1,0.1,0.0
3,"=""00003""",0.0,1.0,0.0,0.0,0.0
4,"=""00004""",0.1,0.1,0.3,0.3,0.2
...,...,...,...,...,...,...
98,"=""00098""",0.1,0.1,0.2,0.4,0.2
99,"=""00099""",0.1,0.4,0.3,0.1,0.1
100,"=""00100""",0.0,1.0,0.0,0.0,0.0
101,"=""00101""",0.1,0.7,0.1,0.0,0.0


In [ ]:
df.to_csv('prediciton_task2_round2.csv',index=False)

In [ ]:
import sys 
import os
import glob
import time
import random
import os
import glob
import time
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Copyright (c) MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
(C) Copyright 2021 IBM Corp.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
   http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
Created on June 30, 2021
"""

from typing import Tuple, Any

import torch.nn as nn
from torch import Tensor
from torch.hub import load_state_dict_from_url
from torchvision.models.video.resnet import VideoResNet, BasicBlock, Conv3DSimple, BasicStem, model_urls


class FuseBackboneResnet3D(VideoResNet):
    """
    3D model classifier (ResNet architecture"
    """

    def __init__(self, pretrained: bool = False, in_channels: int = 1, name: str = "r3d_18") -> None:
        """
        Create 3D ResNet model
        :param pretrained: Use pretrained weights
        :param in_channels: number of input channels
        :param name: model name. currently only 'r3d_18' is supported
        """
        # init parameters per required backbone
        init_parameters = {
            'r3d_18': {'block': BasicBlock,
                       'conv_makers': [Conv3DSimple] * 4,
                       'layers': [2, 2, 2, 2],
                       'stem': BasicStem},
        }[name]
        # init original model
        super().__init__(**init_parameters)

        # load pretrained parameters if required
        if pretrained:
            state_dict = load_state_dict_from_url(model_urls[name])
            self.load_state_dict(state_dict)

        # save input parameters
        self.pretrained = pretrained
        self.in_channels = in_channels
        # override the first convolution layer to support any number of input channels
        self.stem = nn.Sequential(
            nn.Conv3d(self.in_channels, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2),
                      padding=(1, 3, 3), bias=False),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )

    def features(self, x: Tensor) -> Any:
        """
        Extract spatial features - given a 3D tensor
        :param x: Input tensor - shape: [batch_size, channels, z, y, x]
        :return: spatial features - shape [batch_size, n_features, z', y', x']
        """
        x = self.stem(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, x: Tensor) -> Tuple[Tensor, None, None, None]:  # type: ignore
        """
        Forward pass. 3D global classification given a volume
        :param x: Input volume. shape: [batch_size, channels, z, y, x]
        :return: logits for global classification. shape: [batch_size, n_classes].
        """
        x = self.features(x)
        return x
    
backbone=FuseBackboneResnet3D(pretrained=True)   
from typing import Optional, Sequence
import torch.nn as nn

class ClassifierMLP(nn.Module):
    def __init__(self, in_ch: int, num_classes: Optional[int], layers_description: Sequence[int]=(256,128), dropout_rate: float = 0.1):
        super().__init__()
        layer_list = []
        layer_list.append(nn.Linear(in_ch, layers_description[0]))
        layer_list.append(nn.ReLU())
        if dropout_rate is not None and dropout_rate > 0:
            layer_list.append(nn.Dropout(p=dropout_rate))
        last_layer_size = layers_description[0]
        for curr_layer_size in layers_description[1:]:
            layer_list.append(nn.Linear(last_layer_size, curr_layer_size))
            layer_list.append(nn.ReLU())
            if dropout_rate is not None and dropout_rate > 0:
                layer_list.append(nn.Dropout(p=dropout_rate))
            last_layer_size = curr_layer_size
        
        if num_classes is not None:
            layer_list.append(nn.Linear(last_layer_size, num_classes))
        
        self.classifier = nn.Sequential(*layer_list)

    def forward(self, x):
        x = self.classifier(x)
        return x
  
model=ClassifierMLP(12,2)  
import torch    
model=nn.DataParallel(model)
model=model.to(device)

In [ ]:
### testing the model for validation
import pandas as pd

pathtest=pd.read_csv('/content/drive/MyDrive/knight_challenegs2022/test_featurelatest.csv')
#pathtest.

In [ ]:
import torch
pathmodel='/content/drive/MyDrive/knight_challenegs2022/models/fold1model/3dmodeltask1.pth'
trainedmodel=torch.load(pathmodel)
model.load_state_dict(trainedmodel)
model.eval()
model.to(device)

DataParallel(
  (module): ClassifierMLP(
    (classifier): Sequential(
      (0): Linear(in_features=12, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=256, out_features=128, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=128, out_features=2, bias=True)
    )
  )
)

In [ ]:
#NoAT-score
#CanAT-score
#case_id
import torch
pathmodel='/content/drive/MyDrive/knight_challenegs2022/models/fold1model/3dmodeltask1.pth'
trainedmodel=torch.load(pathmodel)
model.load_state_dict(trainedmodel)
model.eval()
model.to(device)
dataframe={'case_id':[],
           'NoAT-score':[],
           'CanAT-score':[]}
CLINICAL_NAMES=['case_id','NoAT-score','CanAT-score']
df = pd.DataFrame(columns=CLINICAL_NAMES)
for i,patinet in enumerate(pathtest['SubjectId']):
  #pat=patinet.split('_')[-1]
  #pat=str(0000)+i
  number_str = str(i)
  pat = number_str.zfill(5)
  #print(i)
  #print(patinet)
  features=pathtest.iloc[i]
  feature=features.drop(['SubjectId'])
  feature1=feature.drop(['gender'])
  #print(feature1.shape)
  x_f_array=np.array(feature1).astype(float)
  #print(x_f_array)
  #.astype(int)
  x_f_array_t=torch.from_numpy(x_f_array).float()
  x_f_array_t=torch.unsqueeze(x_f_array_t,axis=0).to(device)
  prediction=model(x_f_array_t)
  output=torch.softmax(prediction, dim=1)
  output=torch.squeeze(output,axis=0)
  pred=output.detach().cpu().numpy()
  #print(output.detach().cpu().numpy())
  df.loc[i,'case_id']=pat
  df.loc[i,'NoAT-score']='%.1f' % pred[0]
  df.loc[i,'CanAT-score']='%.1f' % pred[1]

  #dataframe['case_id'].append(str(pat))
  #dataframe['NoAT-score'].append(pred[0])
  #dataframe['CanAT-score'].append(pred[1])
  #features.iloc[i]
  #print(x_f_array_t.shape)
  print(pat)

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042
00043
00044
00045
00046
00047
00048
00049
00050
00051
00052
00053
00054
00055
00056
00057
00058
00059
00060
00061
00062
00063
00064
00065
00066
00067
00068
00069
00070
00071
00072
00073
00074
00075
00076
00077
00078
00079
00080
00081
00082
00083
00084
00085
00086
00087
00088
00089
00090
00091
00092
00093
00094
00095
00096
00097
00098
00099
00100
00101
00102


In [ ]:
df['case_id'] = df['case_id'].apply('="{}"'.format)

In [ ]:
df

,case_id,NoAT-score,CanAT-score
0,"=""00000""",0.3,0.7
1,"=""00001""",0.8,0.2
2,"=""00002""",0.8,0.2
3,"=""00003""",1.0,0.0
4,"=""00004""",0.5,0.5
...,...,...,...
98,"=""00098""",0.3,0.7
99,"=""00099""",0.8,0.2
100,"=""00100""",1.0,0.0
101,"=""00101""",0.8,0.2


In [ ]:
df.to_csv('task1_prediction_round2.csv',index=False)